In [ ]:
from braindecode.models import ShallowFBCSPNet, Deep4Net
import torch

n_chans = 27
n_outputs = 2       # MDD vs Healthy
n_times = 640       # e.g. 5s * 128Hz

shallow_model = ShallowFBCSPNet(
    n_chans=n_chans,
    n_outputs=n_outputs,
    n_times=n_times,
    final_conv_length="auto"
)

deep_model = Deep4Net(
    n_chans=n_chans,
    n_outputs=n_outputs,
    n_times=n_times,
    final_conv_length="auto"
)


/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [22]:
device = torch.device("cuda")
shallow_model.to(device)
deep_model.to(device)


Deep4Net(
  (ensuredims): Ensure4d()
  (dimshuffle): Rearrange('batch C T 1 -> batch 1 T C')
  (conv_time_spat): CombinedConv(
    (conv_time): Conv2d(1, 25, kernel_size=(10, 1), stride=(1, 1))
    (conv_spat): Conv2d(25, 25, kernel_size=(1, 64), stride=(1, 1), bias=False)
  )
  (bnorm): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_nonlin): Expression(expression=elu) 
  (pool): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
  (pool_nonlin): Expression(expression=identity) 
  (drop_2): Dropout(p=0.5, inplace=False)
  (conv_2): Conv2d(25, 50, kernel_size=(10, 1), stride=(1, 1), bias=False)
  (bnorm_2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (nonlin_2): Expression(expression=elu) 
  (pool_2): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
  (pool_nonlin_2): Expression(expression=identity) 
  (drop_3): Dropout(p=0.5, inplace=False

In [5]:
import os
from pathlib import Path
import mne
from braindecode.datasets import BaseConcatDataset, BaseDataset
from braindecode.datautil.preprocess import preprocess, Preprocessor
from braindecode.datautil.windowers import create_fixed_length_windows

def prepare_eeg_dataset(data_root, sfreq=128, window_size_s=5, stride_s=5):
    data_root = Path(data_root)
    classes = ['mdd', 'healthy']
    label_map = {'mdd': 0, 'healthy': 1}
    datasets = []

    for label in classes:
        class_path = data_root / label
        fif_files = list(class_path.rglob("*.fif"))
        print(f"Found {len(fif_files)} files in '{label}' folder.")

        for fif_path in fif_files:
            try:
                raw = mne.io.read_raw_fif(fif_path, preload=True)
                raw.pick('eeg')

                if raw.info['nchan'] == 0:
                    print(f"Skipping {fif_path}: no EEG channels found.")
                    continue

                subject_id = fif_path.stem
                dataset = BaseDataset(
                    raw,
                    description={'subject': subject_id, 'label': label_map[label]}
                )
                datasets.append(dataset)

            except Exception as e:
                print(f"Error loading {fif_path}: {e}")
    
    if len(datasets) == 0:
        raise ValueError("No valid EEG datasets loaded.")

    # Combine all datasets
    concat_dataset = BaseConcatDataset(datasets)

    # Preprocessing
    preprocessors = [
        Preprocessor('resample', sfreq=sfreq),
        Preprocessor(lambda x: x * 1e6)  # to µV
    ]
    preprocess(concat_dataset, preprocessors)

    # Windowing
    window_size = int(window_size_s * sfreq)
    stride = int(stride_s * sfreq)

    windows_dataset = create_fixed_length_windows(
        concat_dataset,
        window_size_samples=window_size,
        window_stride_samples=stride,
        drop_last_window=True,
        preload=True
    )

    return windows_dataset


/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/braindecode/datautil/preprocess.py:10: UserWarning: datautil.preprocess module is deprecated and is now under preprocessing.preprocess, please use from import braindecode.preprocessing.preprocess
  warn('datautil.preprocess module is deprecated and is now under '
/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/braindecode/datautil/windowers.py:4: UserWarning: datautil.windowers module is deprecated and is now under preprocessing.windowers, please use from import braindecode.preprocessing.windowers
  warn('datautil.windowers module is deprecated and is now under '


In [7]:
data_dir = "/mnt/data/saikrishna/Team_4/split_fif_new/"  # folder with /mdd and /healthy
windows_ds = prepare_eeg_dataset(data_dir)
print(windows_ds[0])
# # Then do train/val split
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

train_set, test_set = train_test_split(windows_ds, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32)


Found 266 files in 'mdd' folder.
Opening raw data file /mnt/data/saikrishna/Team_4/split_fif_new/mdd/sub-88073797_ses-1_task-restEC_eeg_2_eeg.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 26) active
    Range : 30000 ... 59999 =     60.000 ...   119.998 secs
Ready.
Reading 0 ... 29999  =      0.000 ...    59.998 secs...
Opening raw data file /mnt/data/saikrishna/Team_4/split_fif_new/mdd/sub-88017821_ses-1_task-restEC_eeg_1_eeg.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 26) active
    Range : 0 ... 29999 =      0.000 ...    59.998 secs
Ready.
Reading 0 ... 29999  =      0.000 ...    59.998 secs...
Opening raw data file /mnt/data/saikrishna/Team_4/split_fif_new/mdd/sub-88071949_ses-1_task-restEC_eeg_2_eeg.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 26) active
    Range : 30000 ... 59999 =     60.000 ...   119.998 secs
Ready.
Reading 0 ... 29999  =      0.000 ...    59.998 secs

/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


(array([[ -8.673855  , -10.50176   ,  -8.865224  , ...,   2.3456018 ,
          1.9168735 ,   5.3037233 ],
       [-11.253723  ,  -8.687578  ,  -6.9509244 , ...,  -6.845396  ,
         -1.904621  ,   3.3175907 ],
       [ -0.797503  ,  -7.1749945 ,  -9.375753  , ...,   4.892683  ,
          2.0820584 ,   8.035809  ],
       ...,
       [ -1.3607832 ,   2.1206758 ,  -4.894359  , ...,  23.206337  ,
         16.877916  ,  -2.4171872 ],
       [ -3.3881066 ,  -1.8486235 ,  -3.8903096 , ...,  18.603989  ,
         11.976495  ,  -4.0036626 ],
       [ -4.3474164 ,  -0.40139696,   5.5366    , ...,  10.100729  ,
          7.654722  ,  -0.7284509 ]], dtype=float32), -1, [0, 0, 640])


In [8]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import numpy as np
from braindecode.datasets import BaseConcatDataset

def split_dataset(windows_dataset, batch_size=64, num_workers=4, seed=42):
    np.random.seed(seed)
    description = windows_dataset.get_metadata()
    labels = description['target'].to_numpy()

    train_indices, test_indices = train_test_split(
        np.arange(len(windows_dataset)),
        test_size=0.2,
        stratify=labels,
        random_state=seed
    )

    train_indices, val_indices = train_test_split(
        train_indices,
        test_size=0.2,
        stratify=labels[train_indices],
        random_state=seed
    )

    train_set = Subset(windows_dataset, train_indices)
    val_set = Subset(windows_dataset, val_indices)
    test_set = Subset(windows_dataset, test_indices)
    # Wrap with DataLoaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, val_loader, test_loader


In [9]:
import torch
from braindecode.models import Deep4Net

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Get basic input info
n_channels = windows_ds[0][0].shape[0]
input_window_samples = windows_ds[0][0].shape[1]
n_classes = len(set(windows_ds.get_metadata()['target'].to_list()))

# Initialize Deep4Net model
model = Deep4Net(
    n_chans=n_channels,
    n_outputs=n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto'
).to(device)


/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/braindecode/models/base.py:23: UserWarning: Deep4Net: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [48]:
import os
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

def train_model(model, train_loader, val_loader, epochs=20, lr=0.001):
    model.to(device)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            # Unpack the batch: X = inputs, y = labels
            X = batch[0].to(device)
            y = batch[1].to(device).long()

            # Debugging: print label range and shape
            print(f"Unique label values before adjustment: {y.unique()}")
            y = y - y.min()  # Shift the labels to start from 0
            print(f"Unique label values after adjustment: {y.unique()}")

            # Ensure the correct shape and dtype
            print(f"Output shape: {X.shape}, Label shape: {y.shape}")

            optimizer.zero_grad()
            outputs = model(X)

            # Check if outputs and y have correct shapes
            print(f"Model output shape: {outputs.shape}, Target shape: {y.shape}")

            # Ensure there are no NaNs or Infs in the output
            if torch.isnan(outputs).any() or torch.isinf(outputs).any():
                print("NaN or Inf detected in model output!")
                continue

            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)  # Get the predicted class
            correct += (predicted == y).sum().item()
            total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

        # Optionally validate on val_loader here
        if val_loader is not None:
            evaluate_model(model, val_loader)


def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in loader:
            # Unpack the batch: X = inputs, y = labels
            X = batch[0].to(device)
            y = batch[1].to(device).long()

            # Debugging: print label range and shape
            print(f"Label values: {y.unique()}")
            print(f"Min label: {y.min().item()}, Max label: {y.max().item()}")

            outputs = model(X)
            preds = outputs.argmax(dim=1)  # Get the predicted class
            correct += (preds == y).sum().item()
            total += y.size(0)

    print(f"Validation Accuracy: {correct / total:.4f}")


In [12]:
batch = next(iter(train_loader))
print(type(batch))
print(batch)


<class 'list'>
[tensor([[[ 32.2059,  24.4427,  32.3199,  ...,  19.0148,  30.8760,  36.5537],
         [ 14.9624,  18.4232,  20.7957,  ...,   1.0330,  13.6563,  10.5616],
         [  4.0024,   1.9838,   7.1327,  ...,   8.0362,   8.8394,   8.7511],
         ...,
         [ 11.5652,   6.1531,   3.4537,  ...,  -6.4365,  -9.6270,  -6.9167],
         [ -2.1836,  -3.9191,  -7.5340,  ..., -11.0249, -12.7040,  -9.6289],
         [  2.8010,  -1.5380,  -5.3589,  ...,  -5.0133,  -7.9901,  -5.5692]],

        [[-11.8163,  -8.1658,  -8.7766,  ...,  -2.4287,  -3.1043,  -3.8817],
         [-11.9724, -10.9117, -11.2517,  ...,  -0.2723,  -2.2642,  -2.3579],
         [ -7.1154,  -5.5718,  -7.0899,  ...,   1.1505,  -0.3253,  -3.1006],
         ...,
         [  3.2565,   1.2462,   0.9532,  ...,  -1.6517,  -0.8168,  -3.0834],
         [ 12.7886,  11.8145,  10.7918,  ...,  -8.0116,  -5.3026,  -5.0659],
         [  4.1993,   3.9263,   4.4291,  ...,  -6.7510,  -5.1142,  -2.7930]],

        [[  0.6710,  -7.1897

In [15]:
print(batch[0].shape)

torch.Size([64, 26, 640])


In [ ]:
for batch in train_loader:
    print(f"Type of batch: {type(batch)}")
    print(f"Length of batch: {len(batch)}")
    print(f"Type of batch[0]: {type(batch[0])}")

    # Now safely unpack
    X, y = batch[0], batch[1]
    X = X.to(device)
    y = y.to(device).long()



Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of batch: <class 'list'>
Length of batch: 3
Type of batch[0]: <class 'torch.Tensor'>
Type of ba

NameError: name 'epoch' is not defined

In [38]:
for batch in train_loader:
    # Unpack batch
    X = batch[0]  # The input tensor
    y = batch[1]  # The label tensor

    # Adjust labels if necessary
    y = y + 1  # Change -1 to 0 or another suitable transformation
    
    # Check the shape of X and y
    print(f"X shape: {X.shape}, y shape: {y.shape}")

    # Verify label values
    print(f"Label values: {y.unique()}")
    print(f"Min label: {y.min().item()}, Max label: {y.max().item()}")

    break  # Only process the first batch for now


X shape: torch.Size([64, 26, 640]), y shape: torch.Size([64])
Label values: tensor([0])
Min label: 0, Max label: 0


In [37]:
for batch in train_loader:
    # Unpack batch
    X = batch[0]  # The input tensor
    y = batch[1]  # The label tensor

    # Check the shape of X and y
    print(f"X shape: {X.shape}, y shape: {y.shape}")

    # Verify label values
    print(f"Label values: {y.unique()}")
    print(f"Min label: {y.min().item()}, Max label: {y.max().item()}")

    break  # Only process the first batch for now


X shape: torch.Size([64, 26, 640]), y shape: torch.Size([64])
Label values: tensor([-1])
Min label: -1, Max label: -1


In [50]:
# Inside the train loop
print(f"Label values: {y.unique()}")
print(f"Min label: {y.min().item()}, Max label: {y.max().item()}")


Label values: tensor([1])
Min label: 1, Max label: 1


In [49]:
train_loader, val_loader, test_loader = split_dataset(windows_ds, batch_size=64)
train_model(model, train_loader, val_loader, epochs=20)
evaluate_model(model, test_loader)


# Step 3: Final test evaluation
print("Evaluating on test set:")
evaluate_model(model, test_loader)


Epoch 1/20:   0%|          | 0/53 [00:00<?, ?it/s]

Epoch 1/20:   0%|          | 0/53 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [44]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [45]:
print(f"Unique label values before adjustment: {y.unique()}")
y = y + 1  # Adjust if necessary, and check the labels after adjustment
print(f"Unique label values after adjustment: {y.unique()}")


Unique label values before adjustment: tensor([0])
Unique label values after adjustment: tensor([1])


In [47]:
# %% Imports and Dataset Splitting

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import numpy as np
from braindecode.datasets import BaseConcatDataset

def split_dataset(windows_dataset, batch_size=64, num_workers=4, seed=42):
    """
    Splits the windowed dataset into train, validation, and test sets,
    and returns DataLoaders.
    """
    np.random.seed(seed)
    description = windows_dataset.get_metadata()  # metadata DataFrame with 'target'
    labels = description['target'].to_numpy()

    # Split indices with stratification
    train_indices, test_indices = train_test_split(
        np.arange(len(windows_dataset)),
        test_size=0.2,
        stratify=labels,
        random_state=seed
    )

    train_indices, val_indices = train_test_split(
        train_indices,
        test_size=0.2,
        stratify=labels[train_indices],
        random_state=seed
    )

    # Create subsets using indices
    train_set = Subset(windows_dataset, train_indices)
    val_set = Subset(windows_dataset, val_indices)
    test_set = Subset(windows_dataset, test_indices)

    # Wrap subsets into DataLoaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, val_loader, test_loader

# %% Model Initialization

import torch
from braindecode.models import Deep4Net

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Assume windows_ds is already defined as your Braindecode windowed dataset.
# Extract input information from one sample:
n_channels = windows_ds[0][0].shape[0]         # channels dimension
input_window_samples = windows_ds[0][0].shape[1]  # time samples dimension
n_classes = len(set(windows_ds.get_metadata()['target'].to_list()))

# Initialize the Deep4Net model
model = Deep4Net(
    n_chans=n_channels,
    n_outputs=n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto'
).to(device)

# %% Training and Evaluation Functions

import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

def train_model(model, train_loader, val_loader, epochs=20, lr=1e-3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for X, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            # Move data to device and ensure labels are of type Long
            X, y = X.to(device), y.to(device)
            y = y.long()  # convert to integer type if not already

            optimizer.zero_grad()

            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            correct += (outputs.argmax(dim=1) == y).sum().item()

        acc = correct / len(train_loader.dataset)
        print(f"Train Loss: {running_loss:.4f} | Train Accuracy: {acc:.4f}")

        evaluate_model(model, val_loader)


def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            y = y.long()  # ensure labels are of integer type
            outputs = model(X)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    print(f"Validation Accuracy: {correct / total:.4f}")

# %% Example Usage

# Split the dataset into DataLoaders
train_loader, val_loader, test_loader = split_dataset(windows_ds, batch_size=64)

# Train the model
train_model(model, train_loader, val_loader, epochs=20, lr=1e-3)

# Evaluate on the test set
print("Evaluating on test set:")
evaluate_model(model, test_loader)


/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/braindecode/models/base.py:23: UserWarning: Deep4Net: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
/mnt/data/saikrishna/Team_4/.venv/lib/python3.12/site-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
